In [32]:
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
#Read in health score data
healthScores = pd.read_csv('HealthScores.csv')

#Add a column (feature) to the health score data set for BMI (BMI is a value calculated from weight and height)

#BMI = weight (in kg) / (height (in m))^2
#See https://www.nhs.uk/chq/Pages/how-can-i-work-out-my-bmi.aspx

#copy healthScores data frame into a temporary data frame
temp = healthScores.copy()

#convert weights from lbs to kg
temp['Weight in kg'] = temp['Weight  in lbs'] * 0.453592

#convert heights from inches to metres
temp['Height in m'] = temp['Height in Inch'] * 0.0254

#calculate BMIs and store them in a new column titled "BMI"
healthScores['BMI'] = temp['Weight in kg'] / (temp['Height in m'] **2)

healthScores.head()

,Age,Sex,Weight in lbs,Height in Inch,IQ,Units of alcohol per day,Cigarettes per day,Active,Health Score (high is good),BMI
0,32,Female,101,67,110,0,0,Inactive,251,15.818661
1,47,Female,138,67,84,2,0,Active,193,21.613616
2,25,Male,166,73,106,2,5,Inactive,167,21.900817
3,75,Male,184,73,99,2,0,Inactive,93,24.275605
4,72,Female,105,65,99,2,0,Active,150,17.472721


In [34]:
#rows, columns of table
healthScores.shape

(4999, 10)

In [35]:
#Convert columns containing non-numerical data into numerical data so they can be processed by the support vector machine model

#Sex: Male = 0, Female = 1
#Health Score (high is good): Inactive = 0, Active = 1, Very active = 2

healthScores['Sex'] = healthScores['Sex'].map({'Male': 0, 'Female': 1})
healthScores['Active'] = healthScores['Active'].map({'Inactive': 0, 'Active': 1, 'Very Active': 2})
    
healthScores.head()

,Age,Sex,Weight in lbs,Height in Inch,IQ,Units of alcohol per day,Cigarettes per day,Active,Health Score (high is good),BMI
0,32,1,101,67,110,0,0,0,251,15.818661
1,47,1,138,67,84,2,0,1,193,21.613616
2,25,0,166,73,106,2,5,0,167,21.900817
3,75,0,184,73,99,2,0,0,93,24.275605
4,72,1,105,65,99,2,0,1,150,17.472721


In [36]:
#create a new data frame (x) from the old one, but drop the health score column
#create a new data frame (y) from the health score column

x = healthScores.drop('Health Score (high is good)', axis=1)
y = healthScores['Health Score (high is good)']

y

0       251
1       193
2       167
3        93
4       150
5       120
6       167
7        90
8       181
9       242
10      217
11      124
12      139
13      139
14      264
15      294
16       96
17      109
18       89
19      187
20      183
21       81
22      286
23      109
24      117
25      150
26      137
27      279
28      210
29      127
       ... 
4969    183
4970    228
4971    241
4972     85
4973     94
4974    180
4975    184
4976    137
4977    143
4978     97
4979    177
4980    135
4981    179
4982    156
4983    132
4984    121
4985    130
4986    267
4987    273
4988    190
4989    176
4990    130
4991    109
4992    162
4993    101
4994    133
4995    100
4996    258
4997    125
4998    378
Name: Health Score (high is good), Length: 4999, dtype: int64

In [37]:
x.head()

,Age,Sex,Weight in lbs,Height in Inch,IQ,Units of alcohol per day,Cigarettes per day,Active,BMI
0,32,1,101,67,110,0,0,0,15.818661
1,47,1,138,67,84,2,0,1,21.613616
2,25,0,166,73,106,2,5,0,21.900817
3,75,0,184,73,99,2,0,0,24.275605
4,72,1,105,65,99,2,0,1,17.472721


In [38]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [39]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

x_train

array([[-0.04342542,  0.78926784, -0.46196499, ..., -0.42349441,
         0.42896596,  0.30123965],
       [ 0.01129791, -1.26699702,  1.24274569, ..., -0.42349441,
         0.42896596,  0.85786003],
       [ 1.21521117,  0.78926784, -1.13592038, ...,  1.64030455,
        -0.89902197, -1.7560743 ],
       ...,
       [-0.42648873, -1.26699702,  1.08416795, ...,  3.39453367,
         1.75695388,  0.89671341],
       [ 1.59827448, -1.26699702,  0.25163483, ..., -0.42349441,
        -0.89902197, -1.20349178],
       [-0.20759541, -1.26699702, -0.02587622, ..., -0.42349441,
         1.75695388, -0.35822402]])

In [40]:
from sklearn import svm

SVMModel = svm.SVR()
SVMModel.fit(x_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [41]:
predictions = SVMModel.predict(x_test)
predictions

array([230.68037826, 177.50009328, 115.81743229, ..., 171.91786399,
       137.6676482 , 133.33995711])

In [42]:
#Determine number of accurate predictions - a prediction is deemed to be accurate if it is within 10% of the true health score

actualValues = y_test.values
totalNumValues = len(y_test)

numAccurateResults = 0

for i in range(0, len(predictions)):
    if abs(predictions[i] - actualValues[i]) < (0.1 * healthScores['Health Score (high is good)'].max()):
        numAccurateResults += 1
    
percentAccurateResults = (numAccurateResults / totalNumValues) * 100
percentAccurateResults

93.67999999999999

In [43]:
#Load in and prepare data for 20 individuals for which health score predictions will be made
#Columns need to be made to match healthScores data, and BMI attribute needs to be added

population = pd.read_csv('Population.csv')

#Add a column (feature) to the health score data set for BMI (BMI is a value calculated from weight and height)
temp = population.copy()

#convert weights from lbs to kg
temp['Weight in kg'] = temp['Weight  in lbs'] * 0.453592

#convert heights from inches to metres
temp['Height in m'] = temp['Height in Inch'] * 0.0254

#calculate BMIs and store them in a new column titled "BMI"
population['BMI'] = temp['Weight in kg'] / (temp['Height in m'] **2)

population = population.drop('Person Id', 1)
population = population.drop('Health Score (high is good)', 1)

In [44]:
#Convert columns containing non-numerical data into numerical data so they can be processed by the random forest model

#Sex: Male = 0, Female = 1
#Health Score (high is good): Inactive = 0, Active = 1, Very active = 2

population['Sex'] = population['Sex'].map({'Male': 0, 'Female': 1})
population['Active'] = population['Active'].map({'Inactive': 0, 'Active': 1, 'Very Active': 2})

In [45]:
#Also, normalise the data in each column to improve results

min_max_scaler = preprocessing.MinMaxScaler()

columns = population.columns
#columns = columns.delete(8)

#Formulae for normalising and un-normalising values:
#normalisedVal = (x - min) / (max - min)
#x = (normalisedVal * (max - min)) + min

#for i in range(0, len(columns)):
 #   x_scaled = min_max_scaler.fit_transform(population[[columns[i]]].values.astype(float))
 #   population[columns[i]] = pd.DataFrame(x_scaled)

inputs = scaler.transform(population)
population

,Age,Sex,Weight in lbs,Height in Inch,IQ,Units of alcohol per day,Cigarettes per day,Active,BMI
0,65,0,191,69,104,0,0,1,28.205457
1,46,1,99,66,103,0,0,1,15.978841
2,51,1,135,71,105,1,0,0,18.828470
3,37,1,126,66,96,1,0,1,20.336707
4,77,1,114,67,99,0,0,1,17.854726
5,79,1,100,69,97,1,0,0,14.767255
6,51,1,119,67,103,1,0,1,18.637828
7,41,0,177,70,103,5,0,1,25.396574
8,33,0,185,68,94,0,0,1,28.128842
9,56,1,105,64,95,0,0,2,18.023009


In [46]:
#Perform the predictions for the 20 individuals with unknown health scores, and un-normalise the results

actualPredictions = SVMModel.predict(inputs)
actualPredictions

array([123.35231166, 226.01390073, 144.72007   , 258.33212874,
       154.02667829, 121.96298963, 218.55027763, 147.52397929,
       175.09065003, 233.43121855, 110.5132166 , 172.59642679,
       115.52829427, 283.85941616, 251.91401839, 134.87874442,
       150.09005182,  77.60640497, 131.35102962, 286.1040856 ])

In [47]:
population['Predicted health scores'] = actualPredictions
population

,Age,Sex,Weight in lbs,Height in Inch,IQ,Units of alcohol per day,Cigarettes per day,Active,BMI,Predicted health scores
0,65,0,191,69,104,0,0,1,28.205457,123.352312
1,46,1,99,66,103,0,0,1,15.978841,226.013901
2,51,1,135,71,105,1,0,0,18.828470,144.720070
3,37,1,126,66,96,1,0,1,20.336707,258.332129
4,77,1,114,67,99,0,0,1,17.854726,154.026678
5,79,1,100,69,97,1,0,0,14.767255,121.962990
6,51,1,119,67,103,1,0,1,18.637828,218.550278
7,41,0,177,70,103,5,0,1,25.396574,147.523979
8,33,0,185,68,94,0,0,1,28.128842,175.090650
9,56,1,105,64,95,0,0,2,18.023009,233.431219
